In [1]:
import pickle
import os
import numpy as np
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

def loadData(filename):
    with open(filename, "rb") as f:
        data = pickle.load(f, encoding='latin1')
        return data['data'], data['labels']

2023-06-18 20:25:12.458573: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term
['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
class CifarData:
    def __init__(self, filenames, isShuffle):
        allData = []
        allLabels = []
        for filename in filenames:
            data, labels = loadData(filename)
            allData.append(data)
            allLabels.append(labels)
        self._data = np.vstack(allData)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(allLabels)
        self._size = self._data.shape[0]
        self._startPos = 0
        self._isShuffle = isShuffle
        if isShuffle:
            self.shuffle()
            
    def shuffle(self):
        p = np.random.permutation(self._size)
        self._data = self._data[p]
        self._labels = self._labels[p]

    def nextBatch(self, size):
        endPos = self._startPos + size
        if endPos > self._size:
            if self._isShuffle:
                self.shuffle()
                self._startPos = 0
                endPos = size
            else:
                raise Exception("No more samples")
            
        if endPos > self._size:
            raise Exception("Sample size is less than %d", size)
        
        batch_data = self._data[self._startPos: endPos]
        batch_labels = self._labels[self._startPos: endPos]
        self._startPos = endPos
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, "data_batch_%d" % i) for i in range(1, 6)]
train_data = CifarData(train_filenames, True)
# print(training_data._size)
# print(training_data.nextBatch(10000))

test_filenames = [os.path.join(CIFAR_DIR, "test_batch")]
test_data = CifarData(test_filenames, False)
# print(test_data._size)
# print(test_data.nextBatch(2000))

In [3]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

x_reshape = tf.reshape(x, [-1, 3, 32, 32])
x_image = tf.transpose(x_reshape, perm = [0, 2, 3, 1])

conv1_1 = tf.layers.conv2d(x_image, 32, (3, 3), padding = "same", activation = tf.nn.relu, name = "conv1_1")
conv1_2 = tf.layers.conv2d(conv1_1, 32, (3, 3), padding = "same", activation = tf.nn.relu, name = "conv1_2")
pooling1 = tf.layers.max_pooling2d(conv1_2, (2, 2), (2, 2), name = "pool1")

conv2_1 = tf.layers.conv2d(pooling1, 32, (3, 3), padding = "same", activation = tf.nn.relu, name = "conv2_1")
conv2_2 = tf.layers.conv2d(conv2_1, 32, (3, 3), padding = "same", activation = tf.nn.relu, name = "conv2_2")
pooling2 = tf.layers.max_pooling2d(conv2_2, (2, 2), (2, 2), name = "pool2")

conv3_1 = tf.layers.conv2d(pooling2, 32, (3, 3), padding = "same", activation = tf.nn.relu, name = "conv3_1")
conv3_2 = tf.layers.conv2d(conv3_1, 32, (3, 3), padding = "same", activation = tf.nn.relu, name = "conv3_2")
pooling3 = tf.layers.max_pooling2d(conv3_2, (2, 2), (2, 2), name = "pool3")

flatter = tf.layers.flatten(pooling3)

y_ = tf.layers.dense(flatter, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = y_)

predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with (tf.name_scope("train_op")):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


/var/folders/n4/d8w2s55130vdn6290bjbhblc0000gn/T/ipykernel_9228/4036054297.py:7: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  conv1_1 = tf.layers.conv2d(x_image, 32, (3, 3), padding = "same", activation = tf.nn.relu, name = "conv1_1")
/var/folders/n4/d8w2s55130vdn6290bjbhblc0000gn/T/ipykernel_9228/4036054297.py:8: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  conv1_2 = tf.layers.conv2d(conv1_1, 32, (3, 3), padding = "same", activation = tf.nn.relu, name = "conv1_2")
/var/folders/n4/d8w2s55130vdn6290bjbhblc0000gn/T/ipykernel_9228/4036054297.py:9: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  pooling1 = tf.layers.max_pooling2d(conv1_2, (2, 2), (2, 2), name = "pool1")
/var/folders/n4/d8w2s55130vdn6290bjbhblc0000gn/T

In [4]:
def variable_summary(var, name):
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar("mean", mean)
        tf.summary.scalar("stddev", stddev)
        tf.summary.scalar("max", tf.reduce_max(var))
        tf.summary.scalar("min", tf.reduce_min(var))
        tf.summary.histogram("histogram", var)
        
with tf.name_scope("summary"):
    variable_summary(conv1_1, 'conv1_1')
    variable_summary(conv1_2, 'conv1_2')
    variable_summary(conv2_1, 'conv2_1')
    variable_summary(conv2_2, 'conv2_2')
    variable_summary(conv3_1, 'conv3_1')
    variable_summary(conv3_2, 'conv3_2')

loss_summary = tf.summary.scalar("loss", loss)
accuracy_summary = tf.summary.scalar("accuracy", accuracy)

source_image = (x_image + 1) * 127.5
inputs_summary = tf.summary.image("input_image", source_image)

merged_summary = tf.summary.merge_all()
merged_summary_test = tf.summary.merge([loss_summary, accuracy_summary])

LOG_DIR = "./tensorboard"
run_label = "./run_vgg_tensorboard"
run_dir = os.path.join(LOG_DIR, run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)
train_log_dir = os.path.join(run_dir, "train")
test_log_dir = os.path.join(run_dir, "test")
if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)

In [5]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100
summary_step = 100

with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter(train_log_dir, sess.graph)
    test_writer = tf.summary.FileWriter(test_log_dir)
    fixed_test_data, fix_test_labels = test_data.nextBatch(batch_size)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.nextBatch(batch_size)
        eval_ops = [loss, accuracy, train_op]
        should_log_summary = (i + 1) % summary_step == 0
        if should_log_summary:
            eval_ops.append(merged_summary)
        eval_ops_result = sess.run(
            eval_ops,
            feed_dict = {
                x: batch_data,
                y: batch_labels
            }
        )
        loss_val, acc_val = eval_ops_result[:2]
        
        if should_log_summary:
            train_summary_str = eval_ops_result[-1]
            train_writer.add_summary(train_summary_str, i + 1)
            test_summary_str = sess.run([merged_summary_test], feed_dict = {
                x: fixed_test_data,
                y: fix_test_labels
            })[0]
            test_writer.add_summary(test_summary_str, i + 1)
        
        if (i + 1) % 100 == 0:
            print("[Train] Step: %d, loss: %4.5f, acc: %4.5f" % (i + 1, loss_val, acc_val))
            
        if (i + 1) % 500 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.nextBatch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    }
                )
                all_test_acc_val.append(test_acc_val)
            test_acc_val = np.mean(all_test_acc_val)
            print("[Test ] Step: %d, acc: %4.5f" % (i + 1, test_acc_val))

[Train] Step: 100, loss: 2.27365, acc: 0.10000
[Train] Step: 200, loss: 1.98770, acc: 0.35000
[Train] Step: 300, loss: 1.76688, acc: 0.45000
[Train] Step: 400, loss: 1.38303, acc: 0.40000
[Train] Step: 500, loss: 1.77420, acc: 0.30000
[Test ] Step: 500, acc: 0.43100
[Train] Step: 600, loss: 1.50736, acc: 0.45000
[Train] Step: 700, loss: 1.51901, acc: 0.50000
[Train] Step: 800, loss: 1.60917, acc: 0.50000
[Train] Step: 900, loss: 1.49069, acc: 0.35000
[Train] Step: 1000, loss: 1.53672, acc: 0.45000
[Test ] Step: 1000, acc: 0.50700
[Train] Step: 1100, loss: 1.34513, acc: 0.45000
[Train] Step: 1200, loss: 1.56042, acc: 0.40000
[Train] Step: 1300, loss: 1.51849, acc: 0.45000
[Train] Step: 1400, loss: 1.35350, acc: 0.65000
[Train] Step: 1500, loss: 1.20658, acc: 0.55000
[Test ] Step: 1500, acc: 0.51450
[Train] Step: 1600, loss: 0.94847, acc: 0.65000
[Train] Step: 1700, loss: 0.90943, acc: 0.70000
[Train] Step: 1800, loss: 1.14354, acc: 0.65000
[Train] Step: 1900, loss: 1.24807, acc: 0.45000